## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_denkmalpflegezustaendigkeit**

## Dataset
# **Denkmalpflegezuständigkeit**

## Description

Die Zuständigkeit für Denkmalpflegeobjekte in der Stadt Zürich ist aufgeteilt zwischen dem Amt für Städtebau Zürich und dem Kanton Zürich. Dieser Datensatz zeigt die Zuständigkeitsgebiete in der Stadt Zürich, welche im Wesentlichen den statistischen Quartieren folgen, mit Ausnahme der kantonalen Objekte. Der Datensatz enthält Angaben zu den zuständigen Ansprechpersonen der Bauberatung sowie der Inventarisation der Denkmalpflege des Amtes für Städtebau (Name, Telefonnummer, Email, Präsenzzeiten) sowie Angaben zur Abteilung Denkmalpflege des Kantons Zürich.

**Zweck**: Der Datensatz bildet die Grundlage für eine Suchfunktion auf der Interseite des Amts für Städtebau. Sie ermöglicht es, über die Eingabe der Adresse die Zuständigkeit (Stadt oder Kanton) zu ermitteln und im Fall der Stadt Zürich direkt die entsprechende Ansprechperson zu finden. Die Suchfunktion ist ein Arbeitsinstrument für die Öffentlichkeit, für den Bereich Denkmalpflege des Amts für Städtebau, sowie für die Abteilung Denkmalpflege des Kantons Zürich.

**Genereller Hinweis zum Geodatensatz:**


Weitere Informationen zu diesem Datensatz finden sie unter [«**Denkmalpflegezuständigkeit»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/9bc114a2-67cc-4f0f-8abe-c044e3e679f9).				
			   

**Datenerfassung:**

Der Flächendatensatz basiert auf einem monatlichen Verschnitt der kantonalen Denkmalschutzobjekte mit Daten der amtlichen Vermessung der Stadt Zürich. Änderungen ergeben sich, wenn Denkmalschutzobjekte durch den Kanton Zürich hinzugefügt oder entfernt werden oder wenn Gebiete abweichend von den statistischen Quartieren angepasst werden sollten.

**Datengrundlage:**

Dieser Datenbestand ist von folgenden GeoServer-Datensätzen abhängig:
-	GSRP.KTZH_SDE.DENKMALSCHUTZOBJEKTE_P
-	AV_BO_BOFLAECHE_A
Attribute: gemäss Angaben des Amts für Städtebau, Denkmalpflege

**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/9bc114a2-67cc-4f0f-8abe-c044e3e679f9.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_denkmalpflegezustaendigkeit)

https://www.stadt-zuerich.ch/geodaten/download/Denkmalpflegezustaendigkeit?format=geojson_link<br>


## Metadata
- **Publisher** `Archäologie & Denkmalpflege, Amt für Städtebau, Hochbaudepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Basis­karten,Bauen und Wohnen`
- **Tags** `['amt-fur-stadtebau', 'denkmalpflege', 'denkmalschutzobjekte', 'geodaten', 'geoportal', 'inventar', 'kanton-zurich', 'polygondaten', 'stzh', 'vektordaten', 'zustandigkeit']`
- **Metadata_created** `2023-11-06T03:09:48.170106`
- **Metadata_modified** `2025-09-27T06:33:04.304121`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Denkmalpflegezustaendigkeit?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch